In [1]:
import pickle
import pandas as pd
import numpy as np

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Train/Validation Split
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/train_df.pickle', 'rb') as f:
    train_df = pickle.load(f)
    
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/val_df.pickle', 'rb') as f:
    val_df = pickle.load(f)
    
# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
train_df = train_df[~train_df['Room'].isin([0, 1, 2, 10, 11, 8])]
train_df = train_df.loc[:, train_df.columns != 'No Icon']

# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
val_df = val_df[~val_df['Room'].isin([0, 1, 2, 10, 11, 8])]
val_df = val_df.loc[:, val_df.columns != 'No Icon']


X_train = train_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_train = train_df['Room']

X_val = val_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_val = val_df['Room']

# Classification Algorithms

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


# Define the MNB classifier model
mnb = MultinomialNB()

# Train the model on the training set
mnb.fit(X_train, y_train)

# Predict the target variable on the val set
y_pred = mnb.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.82      0.96      0.88       358
           4       0.70      0.25      0.36       337
           5       0.36      0.96      0.53       632
           6       1.00      0.81      0.89       556
           7       0.43      0.04      0.08       450
           9       0.71      0.03      0.05       382

    accuracy                           0.56      2715
   macro avg       0.67      0.51      0.47      2715
weighted avg       0.66      0.56      0.49      2715

Accuracy: 0.5576427255985267


In [6]:
# Get the predicted probabilities on the validation set
probs = pd.DataFrame(mnb.predict_proba(X_val), columns=mnb.classes_)

# Combine the actual and predicted values with the probabilities
mnb_predictions = pd.DataFrame({'Actual': y_val, 'Predicted': y_pred}).reset_index(drop=True)
probs = pd.concat([mnb_predictions, probs], axis=1)

# Get the maximum probability for each instance
probs_max = probs.iloc[:, 2:].max(axis=1)
probs_max = pd.concat([probs.iloc[:, 0:2], probs_max], axis=1)
probs_max.columns.values[2] = "probability"

# Vary the probability threshold and compute the resulting accuracy
for i in np.arange(0, 1, 0.01):
    all_rows = probs_max[probs_max['probability'] >= i]
    correct = all_rows.query('Actual == Predicted')
    ratio = len(correct) / len(all_rows)
    if ratio >= 0.9:
        print("Ratio:", ratio)
        print("Threshold:", i)
        break

# Compute the accuracy for the threshold of >=i
accuracy_high = len(probs_max.query(f'probability >= {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability >= {i}'))
print(f"Accuracy for threshold >= {i}:", accuracy_high)

# Compute the accuracy for the threshold of <i
accuracy_low = len(probs_max.query(f'probability < {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability < {i}'))
print(f"Accuracy for threshold < {i}:", accuracy_low)

Ratio: 0.9010204081632653
Threshold: 0.5
Accuracy for threshold >= 0.5: 0.9010204081632653
Accuracy for threshold < 0.5: 0.3636887608069164


In [8]:
from sklearn.ensemble import RandomForestClassifier

# Define the random forest classifier model
rfc = RandomForestClassifier()

# Train the model on the training set
rfc.fit(X_train, y_train)

# Predict the target variable on the validation set
y_pred = rfc.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.94       358
           4       0.34      0.88      0.49       337
           5       0.51      0.64      0.57       632
           6       0.92      0.85      0.89       556
           7       0.45      0.16      0.24       450
           9       0.36      0.01      0.02       382

    accuracy                           0.59      2715
   macro avg       0.59      0.58      0.52      2715
weighted avg       0.60      0.59      0.54      2715

Accuracy: 0.5860036832412523


In [10]:
from sklearn.neighbors import KNeighborsClassifier

# Define the k-nearest neighbors classifier model
knn = KNeighborsClassifier()

# Train the model on the training set
knn.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = knn.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.95       358
           4       0.81      0.24      0.37       337
           5       0.50      0.66      0.57       632
           6       0.99      0.83      0.90       556
           7       0.42      0.17      0.24       450
           9       0.24      0.51      0.33       382

    accuracy                           0.57      2715
   macro avg       0.65      0.56      0.56      2715
weighted avg       0.65      0.57      0.57      2715

Accuracy: 0.574585635359116


/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [11]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression classifier model
lr = LogisticRegression(max_iter=1000)

# Train the model on the training set
lr.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = lr.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.95       358
           4       0.62      0.28      0.38       337
           5       0.37      0.96      0.53       632
           6       0.97      0.83      0.89       556
           7       0.49      0.08      0.13       450
           9       0.50      0.00      0.01       382

    accuracy                           0.57      2715
   macro avg       0.65      0.51      0.48      2715
weighted avg       0.64      0.57      0.50      2715

Accuracy: 0.565377532228361


In [13]:
# Predict probabilities for each class on the validation set
probs = pd.DataFrame(lr.predict_proba(X_val), columns=lr.classes_)

# Combine the actual and predicted values with the probabilities
lr_predictions = pd.DataFrame({'Actual': y_val, 'Predicted': lr.predict(X_val)}).reset_index(drop=True)
probs = pd.concat([lr_predictions, probs], axis=1)

# Get the maximum probability for each instance
probs_max = probs.iloc[:, 2:].max(axis=1)
probs_max = pd.concat([probs.iloc[:, 0:2], probs_max], axis=1)
probs_max.columns.values[2] = "probability"

# Vary the probability threshold and compute the resulting accuracy
for i in np.arange(0, 1, 0.01):
    all_rows = probs_max[probs_max['probability'] >= i]
    correct = all_rows.query('Actual == Predicted')
    ratio = len(correct) / len(all_rows)
    if ratio >= 0.9:
        print("Ratio:", ratio)
        print("Threshold:", i)
        break

# Compute the accuracy for the threshold of >=i
accuracy_high = len(probs_max.query(f'probability >= {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability >= {i}'))
print(f"Accuracy for threshold >= {i}:", accuracy_high)

# Compute the accuracy for the threshold of <i
accuracy_low = len(probs_max.query(f'probability < {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability < {i}'))
print(f"Accuracy for threshold < {i}:", accuracy_low)

Ratio: 0.9323770491803278
Threshold: 0.46
Accuracy for threshold >= 0.46: 0.9323770491803278
Accuracy for threshold < 0.46: 0.359401955146636


In [14]:
filename = 'lr_classification_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(lr, file)